In [ ]:
from funcs.notebook import *

from lightkurve import search_lightcurvefile
from altaipony.lcio import from_mast
from altaipony.flarelc import FlareLightCurve


In [ ]:
f = pd.read_csv("../results/2021_08_flares.csv")
gf = f[~f.istart.isnull()]
# ff = ff.drop(["real?","note"], axis=1)
# f = ff.dropna().drop_duplicates(subset=["ID", "qcs","istart"])
gf = gf.groupby(["ID","mission","qcs"]).first().reset_index()[["ID","mission","qcs"]]
gf

In [ ]:
row = gf.iloc[5]


# plot all flares for a light curve at once
ID, qcs = row.ID, int(row.qcs)
sdf = f[(f.qcs == qcs) & (f.ID == ID) & (f.mission == row.mission)]

lc = from_mast(ID,c=qcs,mission=row.mission,cadence="short",flux_type="PDCSAP_FLUX",
               download_dir="/home/ekaterina/Documents/001_science/lcs")

# make it a list of LCs even if only one LC is returned
if type(lc) == FlareLightCurve:

    lc = [lc]

elif type(lc) == list:

    lc = lc
tts = np.array([(l.time[0], l.time[-1]) for l in lc])
print(row.ID, row.qcs)

In [ ]:
%matplotlib inline
for i, row in sdf.iloc[:].iterrows():
    plt.figure(figsize=(17,5))
    ts, tf = row.tstart, row.tstop
    
    # pick
    rel = .8
    if row.mission=="TESS":
        LC = lc[0]
        cad=1.6
    else:
        nlc = np.where((ts>tts)[:,0] != (ts>tts)[:,1])[0][0]
        LC = lc[nlc]
        cad = 2.15
    _ = LC[np.where((LC.time.value>=ts-rel/cad) & (LC.time.value<=tf+rel/cad))]
    plt.plot(_.time.value, _.flux.value, c="k")

    _ = LC[np.where((LC.time.value>=ts-1e-8) & (LC.time.value<=tf+1e-8))]
    plt.scatter(_.time.value, _.flux.value, c="r")
    plt.xlim(ts-rel/cad, ts+rel/cad)
#     plt.xlabel("time [BTJD - 2457000 days]", fontsize=14)
#     plt.ylabel(r"flux [e$^-$ s$^{-1}$]", fontsize=14)
#     plt.savefig("../results/random/SSO_signature_K2_208_s30.png", dpi=300)

In [ ]:
from lightkurve import search_targetpixelfile

In [ ]:
tpf = search_targetpixelfile(ID,quarter=qcs,mission="Kepler").download_all()

In [ ]:
tpf

In [ ]:
tpf[0].interact(notebook_url='localhost:8888')

In [ ]:
lc[0].plot()
# # # plt.scatter(lc[0].time, lc[0].flux)
# plt.xlim(2118,2120)
# plt.ylim(17000,18000);

In [ ]:
1e-3 * 60 * 24

In [ ]:
ff = pd.read_csv("../results/2020_02_flares_vetted.csv")


In [ ]:
_t = 1231.62056693521


select = (ff.tstart > _t - 1e-3) & (ff.tstart < _t + 1e-3)

In [ ]:
ff[select].groupby("note").count()

In [ ]:
ff[select].shape

In [ ]:
ff.loc[select, "note"] = "simultaneous FP in multiple Lcs"

In [ ]:
ff.loc[select].groupby("note").count()

In [ ]:
ff.shape

In [ ]:
ff.to_csv("../results/2020_02_flares_vetted.csv", index=False)

In [ ]:
lc = from_mast("GJ 3270",c=32,mission="TESS",cadence="fast",flux_type="PDCSAP_FLUX",
               download_dir="/home/ekaterina/Documents/001_science/lcs")


In [ ]:
%matplotlib inline
lc.plot()
plt.xlim(2183,2184)
plt.ylim(6300,7000);